<a href="https://colab.research.google.com/github/SreejaThumma/DL_assignment2/blob/main/DL_q2_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets torch

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload Kanye_West.txt


Saving Kanye_West.txt to Kanye_West.txt


In [ ]:
import os
import shutil

os.makedirs("data", exist_ok=True)
shutil.move("Kanye_West.txt", "data/lyrics.txt")


'data/lyrics.txt'

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load GPT-2 and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Use EOS token as pad token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Prepare dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="data/lyrics.txt",
    block_size=128
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Train the model
trainer.train()

# Save model
trainer.save_model("./model")
tokenizer.save_pretrained("./model")


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
100,4.571500
200,4.490900
300,4.254100
400,4.059100
500,4.001700
600,3.836100
700,3.809400


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load fine-tuned model
model_path = "./model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.config.pad_token_id = tokenizer.eos_token_id

# Generate lyrics
def generate_lyrics(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.9,
            top_k=40,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Try it!
print(generate_lyrics("Yeah, I told you once and"))


Yeah, I told you once and she said no
I aint go broke with that bitch again though so Im not sure why it was changed back to me then yo
Like when my ass hurt cause of your depression the doctor gave em an epidural or two like yeah thats cool huh well okay yall gotta say something else soon as they hear ya Yeahhh ok alright Okay if we do this in frontta them please dont kill yourself Ill just hit on another couple more people later tonight uh oh


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load fine-tuned model
model_path = "./model"  # or wherever you saved your model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.config.pad_token_id = tokenizer.eos_token_id

# Function to generate lyrics
def generate_lyrics(prompt="Yeah, I told you once and", max_length=80):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.8,
            top_k=40,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example prompt
print(generate_lyrics("Yeah, I told you once and"))


Yeah, I told you once and we all did it
But now they say this is the way life should be but my soul keeps coming back to me with a vengeance like no other ever had before
Til then Imma give them something new that makes ya wanna feel bad for yourself forever again

For years at least Ill make some money off your ass in debt because thats how people get laid
